In [1]:
import pandas as pd
import json
import numpy as np
import os

# Konfigureerime Pandase sätteid loetavuse huvides
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

# Sisend- ja väljundfailid
INPUT_FILE = 'tehisintellekti-rakendamise-projekt/andmed/toorandmed_aasta.csv'
OUTPUT_FILE = 'tehisintellekti-rakendamise-projekt/andmed/andmed_aasta.csv'


if os.path.exists(INPUT_FILE):
    df = pd.read_csv(INPUT_FILE, low_memory=False)
    print(f"Toorandmed loetud. Andmestiku suurus: {df.shape[0]} rida, {df.shape[1]} veergu.")
else:
    # Kui faili pole, loome testimiseks tühja DataFrame'i või viskame vea
    raise FileNotFoundError(f"Viga: Sisendfaili '{INPUT_FILE}' ei leitud! Kontrolli failiteed.")

# Kiire pilk andmestruktuurile
df.head(2)

print("Samm 1: Teegid laetud ja seadistused tehtud. Andmed sisse loetud.")

Toorandmed loetud. Andmestiku suurus: 3156 rida, 229 veergu.
Samm 1: Teegid laetud ja seadistused tehtud. Andmed sisse loetud.


In [2]:
def resolve_fields(df):
    """
    Loob uued puhtad veerud, eelistades versiooni-põhist infot.
    Tagastab täiendatud DataFrame'i.
    """

    # 1. Veerud, kus ühendame üldise info ja versiooni info (eelistades versiooni)
    # (uus nimi, üldise info veerg, versiooni info veerg)
    merge_mapping = [
        ('nimi_et', 'title__et', 'version__title__et'),
        ('nimi_en', 'title__en', 'version__title__en'),
        ('eap', 'credits', 'version__credits'),
        ('kirjeldus', 'overview__description__et', 'version__overview__description__et')
        #TODO lisa veel tunnuseid

    ]

    # 2. Veerud, mis võetakse ühest konkreetsest kohast (lihtne ümbernimetamine)
    # (uus nimi, vana nimi)
    rename_mapping = [
        ('aine_kood', 'code'),
        ('semester', 'version__target__semester__et'),
        ('oppejoud_json', 'version__participants__lecturers'),
        ('toimumisajad_json', 'version__schedule__entries')
        #TODO lisa veel tunnuseid

    ]

    print("Alustan veergude filtreerimist ja ühendamist...")
    
    for new_col, base, version in merge_mapping:
        base_exists = base in df.columns if base else False
        ver_exists = version in df.columns if version else False
        
        if base_exists and ver_exists:
            df[new_col] = df[version].fillna(df[base])
        elif ver_exists:
            df[new_col] = df[version]
        elif base_exists:
            df[new_col] = df[base]
        else:
            df[new_col] = np.nan
    for new_col, source in rename_mapping:
        if source in df.columns:
            df[new_col] = df[source]
        else:
            df[new_col] = np.nan
    return df


# Rakendame funktsiooni
df_resolved = resolve_fields(df.copy())

# Kontrollime tulemust
print(f"Samm 2: Puhastatud veergude lisamine tehtud.")
print(df_resolved[['aine_kood', 'semester']].isnull().sum())
print(df_resolved['semester'].unique())

Alustan veergude filtreerimist ja ühendamist...
Samm 2: Puhastatud veergude lisamine tehtud.
aine_kood     0
semester     21
dtype: int64
['kevad' 'sügis' nan]


In [3]:
# --- Abifunktsioonid JSON töötlemiseks ---

def parse_json_safe(json_str):
    """
    Teisendab JSON-stringi turvaliselt Pythoni objektiks (list või dict).
    Käsitleb tühje väärtusi (NaN, None) ja katkist JSON-it, tagastades vea korral None.
    """
    if pd.isna(json_str) or json_str == '': 
        return None
    try:
        return json.loads(json_str)
    except (json.JSONDecodeError, TypeError):
        return None


def extract_lecturers(json_str):
    """
    Eraldab JSON-struktuurist õppejõudude nimed ('person_name').
    Tagastab komadega eraldatud stringi unikaalsetest nimedest tähestikulises järjekorras.
    """
    data = parse_json_safe(json_str)
    if not data: return None
    names = [p.get('person_name') for p in data if isinstance(p, dict) and p.get('person_name')]
    return ", ".join(sorted(list(set(names)))) if names else None
 

def extract_schedule_days_et(json_str):
    """
    Analüüsib tunniplaani aegu ja leiab, mis nädalapäevadel aine toimub.
    1. Parsib kuupäeva/kellaaja.
    2. Teisendab päeva nime eesti keelde (nt Monday -> Esmaspäev).
    3. Sorteerib päevad loogilises järjekorras (Esmaspäevast Pühapäevani), mitte tähestiku järgi.
    """
    data = parse_json_safe(json_str)
    if not data: return None
    
    days = set()
    day_map = {
        'Monday': 'Esmaspäev', 'Tuesday': 'Teisipäev', 'Wednesday': 'Kolmapäev',
        'Thursday': 'Neljapäev', 'Friday': 'Reede', 'Saturday': 'Laupäev', 'Sunday': 'Pühapäev'
    }
    
    for entry in data:
        if not isinstance(entry, dict): continue
        time_str = entry.get('time') or entry.get('start_time')
        if time_str:
            dt = pd.to_datetime(time_str, errors='coerce')
            if not pd.isna(dt):
                en_day = dt.day_name()
                days.add(day_map.get(en_day, en_day))
    
    week_order = ['Esmaspäev', 'Teisipäev', 'Kolmapäev', 'Neljapäev', 'Reede', 'Laupäev', 'Pühapäev']
    sorted_days = sorted(list(days), key=lambda d: week_order.index(d) if d in week_order else 99)
    
    return ", ".join(sorted_days) if sorted_days else None


# --- Funktsioonide rakendamine ---
print("Ekstraheerime JSON väljadest infot...")

df_resolved['oppejoud'] = df_resolved['oppejoud_json'].apply(extract_lecturers)
df_resolved['toimumisajad'] = df_resolved['toimumisajad_json'].apply(extract_schedule_days_et)
  
print("Samm 3: JSON töötlemine valmis.")
print(df_resolved[['aine_kood', 'oppejoud', 'toimumisajad']].head(3))

Ekstraheerime JSON väljadest infot...
Samm 3: JSON töötlemine valmis.
     aine_kood  \
0  OIEO.06.046   
1  KKSB.05.092   
2  ARKI.02.030   

                                                                             oppejoud  \
0                                                                           Gea Lepik   
1                                                                    Kadri Medijainen   
2  Jekaterina Nerman, Jürgen Rünk, Pille-Riin Värk, Ragnar Lõivukene, Viljo Kübarsepp   

         toimumisajad  
0                None  
1  Neljapäev, Laupäev  
2                None  


In [4]:
grading_source_cols = [
    ('Miinimumnõuded', 'version__grading__grade_preconditions__et'),
    ('Hindamismeetod', 'version__grading__grade_evaluation__et')
    # lisa veel, kui vaja midagi
]

def combine_grading_info(row):
    return None

df_resolved['hindamine_info'] = df_resolved.apply(combine_grading_info, axis=1)
print("Samm 4: Hindamisinfo koondatud.")

Samm 4: Hindamisinfo koondatud.


In [5]:
final_cols = [
    'aine_kood', 'nimi_et', 'nimi_en', 'eap', 'semester',
    'kirjeldus', 'hindamine_info','toimumisajad','oppejoud'
    #TODO täida
]

existing_cols = [c for c in final_cols if c in df_resolved.columns]
df_final = df_resolved[existing_cols].copy()

print(f"Lõplik ridade arv: {len(df_final)}, lõplik veergude arv: {len(df_final.columns)}")
print("Samm 5: Lõplik andmestik koostatud.")

Lõplik ridade arv: 3156, lõplik veergude arv: 9
Samm 5: Lõplik andmestik koostatud.


In [7]:
df.head()

,course_uuid,code,title__en,version__title__en,credits,overview__description__en,version__overview__description__en,overview__learning_outcomes_text_en,uuid,state__code,state__et,state__en,last_update,title__et,general__input_languages,general__structural_unit_shares,general__type__code,general__type__et,general__type__en,general__structural_units,additional_info__duration_in_semesters,additional_info__assessment_scale__code,additional_info__assessment_scale__et,additional_info__assessment_scale__en,additional_info__is_vota_course,additional_info__is_continuous_learning_course,overview__description__et,overview__objectives,overview__learning_outcomes,_actions__is_editable,_actions__is_version_creatable,_actions__is_deletable,overview__objectives_text_en,version__uuid,version__code,version__state__code,version__state__et,version__state__en,version__last_update,version__title__et,version__credits,version__credit_type,version__target__year__code,version__target__year__et,version__target__year__en,version__target__semester__code,version__target__semester__et,version__target__semester__en,version__target__study_type__code,version__target__study_type__et,version__target__study_type__en,version__target__language__code,version__target__language__et,version__target__language__en,version__target__part__code,version__target__part__et,version__target__part__en,version__target__faculty__code,version__target__faculty__name__en,version__target__faculty__name__et,version__target__faculty__address,version__target__faculty__city,version__target__faculty__level,version__target__faculty__academic,version__target__faculty__supports_continuous_learning,version__target__course_main_structural_unit__code,version__target__course_main_structural_unit__name__en,version__target__course_main_structural_unit__name__et,version__target__course_main_structural_unit__webpage_url,version__target__course_main_structural_unit__address,version__target__course_main_structural_unit__city,version__target__course_main_structural_unit__street,version__target__course_main_structural_unit__zip,version__target__course_main_structural_unit__level,version__target__course_main_structural_unit__academic,version__target__course_main_structural_unit__supports_continuous_learning,version__target__course_main_structural_unit__parent_code,version__target__label__en,version__target__label__et,version__general__year__code,version__general__year__et,version__general__year__en,version__general__input_languages,version__general__structural_unit_shares,version__general__type__code,version__general__type__et,version__general__type__en,version__additional_info__is_vota_course,version__additional_info__is_continuous_learning_course,version__additional_info__study_levels,version__additional_info__hours__lecture,version__additional_info__hours__practice,version__additional_info__hours__seminar,version__additional_info__hours__colloquium,version__additional_info__hours__web_learning,version__additional_info__hours__independent_work,version__additional_info__hours__individual_work,version__additional_info__hours__internship,version__additional_info__is_enlight_course,version__overview__study_languages,version__overview__description__et,version__overview__objectives,version__overview__learning_outcomes,version__grading__independent_work_assessments,version__grading__grade_preconditions__et,version__grading__grade_evaluation__et,version__grading__debt_elimination__et,version__grading__assessment_scale__code,version__grading__assessment_scale__et,version__grading__assessment_scale__en,version__resources__web_based__code,version__resources__web_based__et,version__resources__web_based__en,version__resources__website_url,version__resources__is_moodle_connected,version__resources__learning_environments,version__resources__materials,version__participants__lecturers,version__schedule__weeks__et,version__schedule__entries,version__parent_uuid,version__parent_code,version__parent_credits,version___action

In [6]:
# salvesta fail
df_final.to_csv(OUTPUT_FILE, index=False)
print(f"Samm 6: Andmestik salvestatud faili '{OUTPUT_FILE}'.")

#TODO

PermissionError: [Errno 13] Permission denied: 'tehisintellekti-rakendamise-projekt/andmed/andmed_aasta.csv'